In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from datetime import datetime
import requests
from config import weather_api_key

In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
# Create a set of random latitude/longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs= zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    #if the city is unique, then we will add it to the cities list
    if city not in cities:
            cities.append(city)
#Print the city count to confirm sufficient count.
len(cities)

730

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval    ")
print("----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if(i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint url with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1


# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"] 
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
       # Append the city information into the city_data list
        city_data.append({"City" : city.title(),
                         "Country" : city_country,
                         "Lat" : city_lat,
                         "Lng" : city_lng,
                         "Max Temp" : city_max_temp,
                         "Humidity" : city_humidity,
                         "Cloudiness" : city_clouds,
                         "Wind Speed" : city_wind,
                         "Current Description" : city_description})
# If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete.
print("---------------------------")
print("Data Retrieval Complete    ")
print("---------------------------")


Beginning Data Retrieval    
----------------------------
Processing Record 1 of Set 1 | peterhead
Processing Record 2 of Set 1 | punta arenas
Processing Record 3 of Set 1 | arraial do cabo
Processing Record 4 of Set 1 | chara
Processing Record 5 of Set 1 | plymouth
Processing Record 6 of Set 1 | kapaa
Processing Record 7 of Set 1 | port alfred
Processing Record 8 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 9 of Set 1 | albany
Processing Record 10 of Set 1 | ushuaia
Processing Record 11 of Set 1 | rikitea
Processing Record 12 of Set 1 | mar del plata
Processing Record 13 of Set 1 | sitka
Processing Record 14 of Set 1 | upernavik
Processing Record 15 of Set 1 | kudahuvadhoo
Processing Record 16 of Set 1 | necochea
Processing Record 17 of Set 1 | klaksvik
Processing Record 18 of Set 1 | chipinge
Processing Record 19 of Set 1 | ahuimanu
Processing Record 20 of Set 1 | avarua
Processing Record 21 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 

In [7]:
print(len(city_data))

666


In [8]:
# Convert the array of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Peterhead,GB,57.5058,-1.7981,46.71,76,100,28.72,overcast clouds
1,Punta Arenas,CL,-53.1500,-70.9167,51.91,46,40,28.77,scattered clouds
2,Arraial Do Cabo,BR,-22.9661,-42.0278,71.56,60,75,8.05,broken clouds
3,Chara,RU,56.9064,118.2631,30.51,76,100,2.48,overcast clouds
4,Plymouth,US,41.8334,-70.8328,77.22,62,1,1.01,clear sky
5,Kapaa,US,22.0752,-159.3190,80.58,85,40,1.01,scattered clouds
6,Port Alfred,ZA,-33.5906,26.8910,70.29,81,100,12.19,overcast clouds
7,Albany,US,42.6001,-73.9662,72.75,56,100,1.01,overcast clouds
8,Ushuaia,AR,-54.8000,-68.3000,47.82,75,75,29.93,moderate rain
9,Rikitea,PF,-23.1203,-134.9692,74.43,77,100,21.61,overcast clouds


In [9]:
# Create the output file (CSV)
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")